# 실행 가능성 확인하기

In [1]:
import glob
import os
import tensorflow as tf
import cv2
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential

## 하이퍼파라미터

In [2]:
EPOCHS = 10

DATASET_PATH = 'dataset/2/'
DATASET_OK_PATTERN = DATASET_PATH + 'OK/*.png'
DATASET_FAIL_PATTERN = DATASET_PATH + 'FAIL/*.png'

RESULT_SAVE_PATH = 'results/'

## 단순한 모델 설정

In [17]:
def Model():
    return Sequential([Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)), #tensorflow 버전업으로 코드 변경
                       MaxPool2D(),
                       Conv2D(64, (3, 3), activation='relu'),
                       MaxPool2D(),
                       Conv2D(128, (3, 3), activation='relu'),
                       MaxPool2D(),
                       Conv2D(256, (3, 3), activation='relu'),
                       MaxPool2D(),
                       Flatten(),
                       Dense(1, activation='sigmoid')])

## 데이터셋 불러오기

In [18]:
def preprocess(file_name):
    img = tf.io.read_file(file_name)
    img = tf.image.decode_png(img, channels=1) #tensorflow 버전업으로 코드 변경
    return tf.image.convert_image_dtype(img, tf.float32)

In [19]:
ok_list = glob.glob(DATASET_OK_PATTERN)
ds_ok = tf.data.Dataset.list_files(ok_list)
ds_ok_label = tf.data.Dataset.from_tensor_slices([0] * len(ok_list))

ds_ok = ds_ok.map(preprocess)
ds_ok = tf.data.Dataset.zip((ds_ok, ds_ok_label))

fail_list = glob.glob(DATASET_FAIL_PATTERN)
ds_fail = tf.data.Dataset.list_files(fail_list)
ds_fail_label = tf.data.Dataset.from_tensor_slices([1] * len(fail_list))

ds_fail = ds_fail.map(preprocess)
ds_fail = tf.data.Dataset.zip((ds_fail, ds_fail_label))

ds = tf.data.Dataset.concatenate(ds_ok, ds_fail)

## Train, Valid 데이터셋 나누기

In [20]:
ds_size = len(ok_list) + len(fail_list)
train_size = int(ds_size * 0.7)

ds = ds.shuffle(ds_size)
ds_train = ds.take(train_size).shuffle(1024, reshuffle_each_iteration=True).batch(32)
ds_valid = ds.skip(train_size).batch(32)

## 모델 생성 및 학습

In [21]:
model = Model()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [22]:
model.fit(ds_train, validation_data=ds_valid, epochs=EPOCHS)

Epoch 1/10
21/21 [==============================] - 33s 2s/step - loss: 0.4329 - accuracy: 0.8744 - val_loss: 0.4156 - val_accuracy: 0.8521
Epoch 2/10
21/21 [==============================] - 30s 1s/step - loss: 0.3786 - accuracy: 0.8669 - val_loss: 0.3198 - val_accuracy: 0.8908
Epoch 3/10
21/21 [==============================] - 30s 1s/step - loss: 0.3963 - accuracy: 0.8714 - val_loss: 0.3016 - val_accuracy: 0.9049
Epoch 4/10
21/21 [==============================] - 35s 2s/step - loss: 0.3375 - accuracy: 0.8820 - val_loss: 0.3599 - val_accuracy: 0.8521
Epoch 5/10
21/21 [==============================] - 34s 2s/step - loss: 0.3446 - accuracy: 0.8744 - val_loss: 0.4015 - val_accuracy: 0.8873
Epoch 6/10
21/21 [==============================] - 34s 2s/step - loss: 0.3644 - accuracy: 0.8835 - val_loss: 0.4414 - val_accuracy: 0.8380
Epoch 7/10
21/21 [==============================] - 33s 2s/step - loss: 0.3632 - accuracy: 0.8805 - val_loss: 0.3497 - val_accuracy: 0.8732
Epoch 8/10
21/21 [==

## 결과를 이미지로 저장

In [23]:
def mkdir(path):
    if os.path.exists(path) is False:
        os.mkdir(path)

mkdir(RESULT_SAVE_PATH)
mkdir(RESULT_SAVE_PATH + '/TP')
mkdir(RESULT_SAVE_PATH + '/TN')
mkdir(RESULT_SAVE_PATH + '/FP')
mkdir(RESULT_SAVE_PATH + '/FN')

index = 0
for imgs, labels in ds_valid:
    preds = model.predict(imgs)
    for idx in range(imgs.shape[0]):
        gt = labels[idx].numpy()
        y = preds[idx]
        
        if gt == 1 and y > 0.5:
            path = RESULT_SAVE_PATH + '/TP'
        elif gt == 1 and y <= 0.5:
            path = RESULT_SAVE_PATH + '/FN'
        elif gt == 0 and y > 0.5:
            path = RESULT_SAVE_PATH + '/FP'
        else:
            path = RESULT_SAVE_PATH + '/TN'
            
        cv2.imwrite(path + '/%.4f_%04d.png' % (y, index), imgs[idx].numpy() * 255)
        index +=1